# Test


In [ ]:
import sqlite3 
import os
import pandas as pd
import glob

### "Pre processing" the data 


In [ ]:
%pip install openpyxl

In [ ]:
df = pd.read_excel('content\ground_truth\data.xlsx')
ground_truth_df = df[['Study_ID', 'Study', 'Allocation', 'Experimenter']]
ground_truth_df

In [ ]:
# extracting data from the csv files in the extracted directory

extracted_dir = 'content/extracted'
extracted_dfs = []

#def extract_data_from_csv(extracted_dir, extracted_dfs):
if os.path.exists(extracted_dir):
    for filename in os.listdir(extracted_dir):
        if filename.endswith('.csv'):
            file_path = os.path.join(extracted_dir, filename)
            try:
                df = pd.read_csv(file_path)
                extracted_dfs.append(df)
                print(f"Loaded {filename}")
            except Exception as e:
                print(f"Failed to load {filename}: {e}")

if extracted_dfs:
    extracted_combined_df = pd.concat(extracted_dfs, ignore_index=True)
    print("Combined DataFrame created.")
else:
    print("No CSV files found in the directory.")

In [34]:
def extract_data_from_csv(extracted_dir):
    """
    Loads all CSV files from the given directory into a combined DataFrame.

    Args:
        extracted_dir (str): Path to the directory containing CSV files.

    Returns:
        pd.DataFrame: Combined DataFrame of all loaded CSV files.
    """
    print(f"Extracting data from CSV files in directory: {extracted_dir}")
    if os.path.exists(extracted_dir):
        for filename in os.listdir(extracted_dir):
            if filename.endswith('.csv'):
                file_path = os.path.join(extracted_dir, filename)
                try:
                    df = pd.read_csv(file_path)
                    extracted_dfs.append(df)
                    #print(f"Loaded {filename}")
                except Exception as e:
                    print(f"Failed to load {filename}: {e}")

    if extracted_dfs:
        extracted_combined_df = pd.concat(extracted_dfs, ignore_index=True)
        print("Combined DataFrame created.")
        print(f"Total rows in combined DataFrame: {len(extracted_combined_df)}")
        print("-"* 40)
        return extracted_combined_df
    else:
        print("No CSV files found in the directory.")
        return pd.DataFrame()

In [37]:
extracted_combined_df = extract_data_from_csv(extracted_dir)

Extracting data from CSV files in directory: content/extracted
Combined DataFrame created.
Total rows in combined DataFrame: 472
----------------------------------------


In [ ]:
ext_split1_shuffle_1 = extract_data_from_csv('content\extracted_split_1_shuffle_1')
ext_split1_shuffle_2 = extract_data_from_csv('content\extracted_split_1_shuffle_2')
ext_split2_shuffle_1 = extract_data_from_csv('content\extracted_split_2_shuffle_1')
ext_split2_shuffle_2 = extract_data_from_csv('content\extracted_split_2_shuffle_2')
ext_split3_shuffle_1 = extract_data_from_csv('content\extracted_split_3_shuffle_1')
ext_split3_shuffle_2 = extract_data_from_csv('content\extracted_split_3_shuffle_2')
ext_split4_shuffle_1 = extract_data_from_csv('content\extracted_split_4_shuffle_1')
ext_split4_shuffle_2 = extract_data_from_csv('content\extracted_split_4_shuffle_2')

Extracting data from CSV files in directory: content\extracted_split_1_shuffle_1
Combined DataFrame created.
Total rows in combined DataFrame: 339
----------------------------------------
Extracting data from CSV files in directory: content\extracted_split_1_shuffle_2
Combined DataFrame created.
Total rows in combined DataFrame: 357
----------------------------------------
Extracting data from CSV files in directory: content\extracted_split_2_shuffle_1
Combined DataFrame created.
Total rows in combined DataFrame: 374
----------------------------------------
Extracting data from CSV files in directory: content\extracted_split_2_shuffle_2
Combined DataFrame created.
Total rows in combined DataFrame: 391
----------------------------------------
Extracting data from CSV files in directory: content\extracted_split_3_shuffle_1
Combined DataFrame created.
Total rows in combined DataFrame: 407
----------------------------------------
Extracting data from CSV files in directory: content\extract

In [38]:
# directly taken from Anna's code
import re
import unicodedata

def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    return "".join([c for c in nfkd_form if not unicodedata.combining(c)])

def format_studyName(study_name_string):
    """
    Cuts a string after the last four-digit number, assuming it represents the year.

    Args:
        study_name_string (str): The input string potentially containing a year.

    Returns:
        str: The string cut after the year, or the original string if no year is found.
    """
    # Get rid of all the points, -
    study_name_string = study_name_string.replace('.', '')
    study_name_string = study_name_string.replace(',', '')
    study_name_string = study_name_string.replace(' - ', ' ')
    study_name_string = study_name_string.replace(')', '')
    study_name_string = study_name_string.replace('(', '')
    study_name_string = study_name_string.replace('&', 'and')
    study_name_string = remove_accents(study_name_string)
    # Find all occurrences of four consecutive digits (potential years)
    year_matches = list(re.finditer(r'\b\d{4}\b', study_name_string))

    if year_matches:
        # Get the last match
        last_year_match = year_matches[-1]
        # Get the end index of the last year match
        end_of_year_index = last_year_match.end()
        # Slice the string up to the end of the year
        cut_string = study_name_string[:end_of_year_index]
        return cut_string.strip() # Use strip to remove trailing whitespace
    else:
        # If no four-digit number is found, return the original string
        return study_name_string.strip()

In [43]:
extracted_combined_df['Study'] = extracted_combined_df['Study'].apply(format_studyName)
extracted_combined_df

ext_split1_shuffle_1['Study'] = ext_split1_shuffle_1['Study'].apply(format_studyName)
ext_split1_shuffle_2['Study'] = ext_split1_shuffle_2['Study'].apply(format_studyName)
ext_split2_shuffle_1['Study'] = ext_split2_shuffle_1['Study'].apply(format_studyName)
ext_split2_shuffle_2['Study'] = ext_split2_shuffle_2['Study'].apply(format_studyName)
ext_split3_shuffle_1['Study'] = ext_split3_shuffle_1['Study'].apply(format_studyName)
ext_split3_shuffle_2['Study'] = ext_split3_shuffle_2['Study'].apply(format_studyName)
ext_split4_shuffle_1['Study'] = ext_split4_shuffle_1['Study'].apply(format_studyName)
ext_split4_shuffle_2['Study'] = ext_split4_shuffle_2['Study'].apply(format_studyName)


#### To group the tables by Study

In [45]:
#provided by Anna 
def accuracy_check(col_name, df_extracted, test_table):
  allocation_match = False
  experimenter_match = False

  # Check if both dataframes have the expected columns and rows
  if col_name in df_extracted.columns and \
    not df_extracted.empty and not test_table.empty:

      extracted_allocation = df_extracted[col_name].iloc[0]

      ground_truth_allocation = test_table[col_name].iloc[0]

      # Simple case-insensitive comparison
      if str(extracted_allocation).lower() == str(ground_truth_allocation).lower():
          allocation_match = True
          print(f'{col_name}: Match')
      else:
          print(f"{col_name}: Mismatch (Extracted: '{extracted_allocation}', Ground Truth: '{ground_truth_allocation}')")
  else:
    print("Cannot perform accuracy check: Extracted or ground truth data is missing or malformed.")
  print("--------------------")

In [46]:
# Use the accuracy_check function to compare 'Allocation' between df and extracted_combined_df
accuracy_check('Allocation', extracted_combined_df, ground_truth_df)
accuracy_check('Experimenter', extracted_combined_df, ground_truth_df)

accuracy_check('Allocation', ext_split1_shuffle_1, ground_truth_df)
accuracy_check('Experimenter', ext_split1_shuffle_1, ground_truth_df)
accuracy_check('Allocation', ext_split1_shuffle_2, ground_truth_df)
accuracy_check('Experimenter', ext_split1_shuffle_2, ground_truth_df)

accuracy_check('Allocation', ext_split2_shuffle_1, ground_truth_df)
accuracy_check('Experimenter', ext_split2_shuffle_1, ground_truth_df)
accuracy_check('Allocation', ext_split2_shuffle_2, ground_truth_df)
accuracy_check('Experimenter', ext_split2_shuffle_2, ground_truth_df)

accuracy_check('Allocation', ext_split3_shuffle_1, ground_truth_df)
accuracy_check('Experimenter', ext_split3_shuffle_1, ground_truth_df)
accuracy_check('Allocation', ext_split3_shuffle_2, ground_truth_df)
accuracy_check('Experimenter', ext_split3_shuffle_2, ground_truth_df)

accuracy_check('Allocation', ext_split4_shuffle_1, ground_truth_df)
accuracy_check('Experimenter', ext_split4_shuffle_1, ground_truth_df)
accuracy_check('Allocation', ext_split4_shuffle_2, ground_truth_df)
accuracy_check('Experimenter', ext_split4_shuffle_2, ground_truth_df)


Allocation: Match
--------------------
Experimenter: Mismatch (Extracted: 'Teacher', Ground Truth: 'Parent')
--------------------
Allocation: Match
--------------------
Experimenter: Mismatch (Extracted: 'Teacher', Ground Truth: 'Parent')
--------------------
Allocation: Match
--------------------
Experimenter: Mismatch (Extracted: 'Teacher', Ground Truth: 'Parent')
--------------------
Allocation: Match
--------------------
Experimenter: Mismatch (Extracted: 'Teacher', Ground Truth: 'Parent')
--------------------
Allocation: Match
--------------------
Experimenter: Mismatch (Extracted: 'Teacher', Ground Truth: 'Parent')
--------------------
Allocation: Match
--------------------
Experimenter: Mismatch (Extracted: 'Teacher', Ground Truth: 'Parent')
--------------------
Allocation: Match
--------------------
Experimenter: Mismatch (Extracted: 'Teacher', Ground Truth: 'Parent')
--------------------
Allocation: Match
--------------------
Experimenter: Mismatch (Extracted: 'Teacher', Groun

## Calculating the metrics
Auxilary functions used to calculate the metrics just so it can be easier to read

In [ ]:
# Function to calculate various metrics based on true positives, false positives, and false negatives
def calculateAccuracy(TP, FP, FN):
    '''calculates the accuracy of a model based on true positives, true negatives, false positives, and false negatives.'''
    return float(TP + / (TP + FP + FN) if (TP + FP + FN) > 0 else 0)

def calculatePrecision(TP, FP):
    '''calculates the precision of a model based on true positives and false positives.'''
    return float(TP / (TP + FP) if (TP + FP) > 0 else 0)

def calculateRecall(TP, FN):
    '''calculates the recall of a model based on true positives and false negatives.'''
    return float(TP / (TP + FN) if (TP + FN) > 0 else 0)

def calculateF1Score(precision, recall):
    '''calculates the F1 score based on precision and recall.'''
    return float(2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0)

def calculateMetrics(TP, FP, FN):
    '''calculates various metrics based on true positives, false positives, and false negatives.'''
    # --- Metrics ---
    accuracy = calculateAccuracy(TP, FP, FN) 
    recall = calculateRecall(TP, FN)
    precision = calculatePrecision(TP, FP)
    f1 = calculateF1Score(precision, recall)

    return {
        'Accuracy': round(accuracy, 4),
        'Recall': round(recall, 4),
        'Precision': round(precision, 4),
        'F1': round(f1, 4)
    }


In [ ]:
# Function to count true positives, false positives, and false negatives
def count_true_positives(gt_df, pred_df, column='Allocation', join_on='Study', positive_value='Random'):
    """
    Counts the number of true positives in the ground truth and predicted dataframes.
    Args:
        gt_df (pd.DataFrame): Ground truth dataframe.
        pred_df (pd.DataFrame): Predicted dataframe.
        column (str): The column name to check for true positives.
        join_on (str): The column name to join the dataframes on.
        positive_value (str): The value considered as a positive case.
    Returns:
        int: The count of true positives.
    """
    merged = gt_df[[join_on, column]].merge(pred_df[[join_on, column]], on=join_on, suffixes=('_gt', '_pred'))
    tp = ((merged[f"{column}_gt"] == positive_value) & (merged[f"{column}_pred"] == positive_value)).sum()
    return tp

def count_false_positives(gt_df, pred_df, column='Allocation', join_on='Study', positive_value='Random'):
    """Counts the number of false positives in the ground truth and predicted dataframes.
    Args:
        gt_df (pd.DataFrame): Ground truth dataframe.
        pred_df (pd.DataFrame): Predicted dataframe.
        column (str): The column name to check for false positives.
        join_on (str): The column name to join the dataframes on.
        positive_value (str): The value considered as a positive case.
    Returns:
        int: The count of false positives.
    """
    merged = gt_df[[join_on, column]].merge(pred_df[[join_on, column]], on=join_on, suffixes=('_gt', '_pred'))
    fp = ((merged[f"{column}_gt"] != positive_value) & (merged[f"{column}_pred"] == positive_value)).sum()
    return fp

def count_false_negatives(gt_df, pred_df, column= 'Allocation', join_on='Study', positive_value='Random'):
    """Counts the number of false negatives in the ground truth and predicted dataframes.
    Args:
        gt_df (pd.DataFrame): Ground truth dataframe.
        pred_df (pd.DataFrame): Predicted dataframe.
        column (str): The column name to check for false negatives.
        join_on (str): The column name to join the dataframes on.
        positive_value (str): The value considered as a positive case.
    Returns:
        int: The count of false negatives."""
    
    merged = gt_df[[join_on, column]].merge(pred_df[[join_on, column]], on=join_on, suffixes=('_gt', '_pred'))
    fn = ((merged[f"{column}_gt"] == positive_value) & (merged[f"{column}_pred"] != positive_value)).sum()
    return fn

### Testing the results :
This will probably be deleted or reorgenised to keep the data in a better format for future plots

In [ ]:
tp = count_true_positives(ground_truth_df, extracted_combined_df, column='Allocation', join_on='Study', positive_value='Random')
print(f'True Positives: {tp}')

fp = count_false_positives(ground_truth_df, extracted_combined_df, column='Allocation', join_on='Study', positive_value='Random')
print(f'False Positives: {fp}')

fn = count_false_negatives(ground_truth_df, extracted_combined_df, column='Allocation', join_on='Study', positive_value='Random')
print(f'False Negatives: {fn}')

# Calculate metrics
allocation_metrics = calculateMetrics(tp, fp, fn)
print(allocation_metrics)


In [ ]:
tp_teacher = count_true_positives(ground_truth_df, extracted_combined_df, column='Experimenter', join_on='Study', positive_value='Teacher')
print(f'True Positives (Teacher): {tp_teacher}')

fp_teacher = count_false_positives(ground_truth_df, extracted_combined_df, column='Experimenter', join_on='Study', positive_value='Teacher')
print(f'False Positives (Teacher): {fp_teacher}')

fn_teacher = count_false_negatives(ground_truth_df, extracted_combined_df, column='Experimenter', join_on='Study', positive_value='Teacher')
print(f'False Negatives (Teacher): {fn_teacher}')

# Calculate metrics for Teacher
metrics_teacher = calculateMetrics(tp_teacher, fp_teacher, fn_teacher)
print(metrics_teacher)

In [ ]:
tp_parent = count_true_positives(ground_truth_df, extracted_combined_df, column='Experimenter', join_on='Study', positive_value='Parent')
print(f'True Positives (Parent): {tp_parent}')

fp_parent = count_false_positives(ground_truth_df, extracted_combined_df, column='Experimenter', join_on='Study', positive_value='Parent')
print(f'False Positives (Parent): {fp_parent}')

fn_parent = count_false_negatives(ground_truth_df, extracted_combined_df, column='Experimenter', join_on='Study', positive_value='Parent')
print(f'False Negatives (Parent): {fn_parent}')

metrics_parent = calculateMetrics(tp_parent, fp_parent, fn_parent)
print(metrics_parent)

In [ ]:
tp_researcher = count_true_positives(ground_truth_df, extracted_combined_df, column='Experimenter', join_on='Study', positive_value='Researcher')
print(f'True Positives (Researcher): {tp_researcher}')

fp_researcher = count_false_positives(ground_truth_df, extracted_combined_df, column='Experimenter', join_on='Study', positive_value='Researcher')
print(f'False Positives (Researcher): {fp_researcher}')

fn_researcher = count_false_negatives(ground_truth_df, extracted_combined_df, column='Experimenter', join_on='Study', positive_value='Researcher')
print(f'False Negatives (Researcher): {fn_researcher}')

metrics_researcher = calculateMetrics(tp_researcher, fp_researcher, fn_researcher)
print(metrics_researcher)

In [ ]:
tp_Combined = count_true_positives(ground_truth_df, extracted_combined_df, column='Experimenter', join_on='Study', positive_value='Combined - Teacher and Parent')
print(f'True Positives (Combined): {tp_Combined}')

fp_Combined = count_false_positives(ground_truth_df, extracted_combined_df, column='Experimenter', join_on='Study', positive_value='Combined - Teacher and Parent')
print(f'False Positives (Combined): {fp_Combined}')

fn_Combined = count_false_negatives(ground_truth_df, extracted_combined_df, column='Experimenter', join_on='Study', positive_value='Combined - Teacher and Parent')
print(f'False Negatives (Combined): {fn_Combined}')

metrics_Combined = calculateMetrics(tp_Combined, fp_Combined, fn_Combined)
print(metrics_Combined)

In [ ]:
metrics_Experimenter = calculateMetrics(tp_teacher + tp_parent + tp_researcher + tp_Combined,
                                        fp_teacher + fp_parent + fp_researcher + fp_Combined,
                                        fn_teacher + fn_parent + fn_researcher + fn_Combined)
print("Experimenter : ", metrics_Experimenter)

# Plots 
plotting the outputted numbers uwu

In [ ]:
%pip install matplotlib
import matplotlib.pyplot as plt

In [ ]:
def bar_plot_allocation_metrics(metrics_dicts, model_names):
    """
    Plots allocation metrics (Accuracy, Recall, Precision, F1) for different LLMs.

    Args:
        metrics_dicts (list of dict): List of metrics dictionaries, one per model.
        model_names (list of str): List of model names corresponding to metrics_dicts.
    """
    import matplotlib.pyplot as plt

    labels = ['Accuracy', 'Recall', 'Precision', 'F1']
    x = range(len(labels))
    colors = ['hotpink', '#88c999', 'purple', "#92e6df",'#e15759', '#edc949', ]

    plt.figure(figsize=(10, 6))
    width = 0.8 / len(metrics_dicts)  # Adjust bar width based on number of models

    for i, (metrics, name) in enumerate(zip(metrics_dicts, model_names)):
        values = [metrics[label] for label in labels]
        plt.bar([xi + i * width for xi in x], values, width=width, label=name, color=colors[i % len(colors)])

    plt.ylabel('Score')
    plt.title('Allocation and Experimenter metrics (ChatGPT output)')
    plt.xticks([xi + width * (len(metrics_dicts) - 1) / 2 for xi in x], labels)
    plt.ylim(0, 1)
    plt.legend()
    #plt.tight_layout()
    plt.show()

In [ ]:
bar_plot_allocation_metrics([allocation_metrics, metrics_Experimenter], ['Allocation', 'Experimenter'])

In [ ]:
# Test plot_allocation_metrics with a single model
bar_plot_allocation_metrics([ metrics_parent, metrics_researcher, metrics_teacher, metrics_Combined], ['Parent', 'Researcher', 'Teacher', 'Combined'])

In [ ]:
def scatter_plot_accuracy_vs_time(accuracies, computing_times, model_names):
    """
    Plots a scatter plot of accuracy vs computing time for each LLM.

    Args:
        accuracies (list): List of accuracy values for each model.
        computing_times (list): List of computing times for each model.
        model_names (list): List of model names (e.g., ['chatgpt', 'llama', 't5']).
    """

    default_colors = ['hotpink', '#88c999', 'purple', 'blue', "#92e6df", '#e15759', '#edc949']
    colors = default_colors * ((len(accuracies) // len(default_colors)) + 1)
    colors = colors[:len(accuracies)]

    plt.figure(figsize=(8, 6))
    plt.scatter(computing_times, accuracies, color= colors, s=100)

    for i, name in enumerate(model_names):
        plt.annotate(name, (computing_times[i], accuracies[i]), textcoords="offset points", xytext=(5,5), ha='left')

    plt.xlabel('Computing Time (s)')
    plt.ylabel('Accuracy')
    plt.title('Accuracy vs Computing Time for LLMs')
    plt.grid(True)
    plt.show()

In [ ]:
# Test the scatter_plot_accuracy_vs_time function with available data
scatter_plot_accuracy_vs_time([0.3, 0.4, 0.896, 1], [5, 3, 0.5, 10], ['model_names', 'gpt', 'llama', 'test'])

# To do :
nexts steps are to add graphics
- multiple subplots on a single page side by side